## Load/import packages

In [22]:
import scipy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential, layers
from sklearn.utils import class_weight


# Import from features, labels and reshaper function
from load_features import (
    train_features,
    val_features,
    train_labels,
    val_labels,
    reshaper,
)

from functions import f1, plot_history, arr_replacevalue

%matplotlib inline

# Check if Tensorflow uses GPU
print(tf.config.experimental.list_physical_devices("GPU"))

# Limit GPU memory usage
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print()
print(f"Tensorflow Version: {tf.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Tensorflow Version: 2.4.0-rc0
Numpy Version: 1.19.2
Keras Version: 2.4.0


# Prepare data

In [3]:
# Reshape data to sequence length 150
seq_train_features, seq_train_labels = reshaper(train_features, train_labels, 150)
seq_val_features, seq_val_labels = reshaper(val_features, val_labels, 150)

inp_shape = (seq_train_features.shape[1], seq_train_features.shape[2])

In [4]:
# Convert one-hot encoded labels back to label integers
label_ints = np.argmax(seq_train_labels, axis=2)

# Compute class weights with sklearn
class_weights = class_weight.compute_class_weight(
    "balanced", np.unique(label_ints), label_ints.flatten()
)
d_class_weights = dict(enumerate(class_weights))

# Copy label integer array
arr = label_ints.copy()

# Pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample
samples_weights = arr_replacevalue(arr, d_class_weights)

C:\Users\ion\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4 5 6], y=[0 0 0 ... 6 6 6] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


# Build model

In [17]:
class parameters():

    def __init__(self):

        self.input_dim = 150
        self.num_classes = 7
        self.num_epochs = 1000
        self.batch_size = 32

        self.num_hidden_units = 50
        self.l = 0.95 # decay lambda
        self.e = 0.5 # learning rate eta
        self.S = 1 # num steps to get to h_S(t+1)
        self.learning_rate = 1e-4
        self.learning_rate_decay_factor = 0.99 # don't use this decay
        self.max_gradient_norm = 5.0

In [50]:
class FW_RNN(object):

    def __init__(self, FLAGS):

        self.X = tf.keras.Input(tf.float32,
            shape=(None, 150, 4608))
        self.y = tf.keras.Input(tf.float32,
            shape=(None, 150, 7))
        self.l = tf.keras.Input(tf.float32, [], # need [] for tf.scalar_mul
            name="learning_rate")
        self.e = tf.keras.Input(tf.float32, [],
            name="decay_rate")

        with tf.variable_scope("fast_weights"):

            # input weights (proper initialization)
            self.W_x = tf.Variable(tf.random_uniform(
                [FLAGS.num_classes, FLAGS.num_hidden_units],
                -np.sqrt(2.0/FLAGS.num_classes),
                np.sqrt(2.0/FLAGS.num_classes)),
                dtype=tf.float32)
            self.b_x = tf.Variable(tf.zeros(
                [FLAGS.num_hidden_units]),
                dtype=tf.float32)

            # hidden weights (See Hinton's video @ 21:20)
            self.W_h = tf.Variable(
                initial_value=0.05 * np.identity(FLAGS.num_hidden_units),
                dtype=tf.float32)

            # softmax weights (proper initialization)
            self.W_softmax = tf.Variable(tf.random_uniform(
                [FLAGS.num_hidden_units, FLAGS.num_classes],
                -np.sqrt(2.0 / FLAGS.num_hidden_units),
                np.sqrt(2.0 / FLAGS.num_hidden_units)),
                dtype=tf.float32)
            self.b_softmax = tf.Variable(tf.zeros(
                [FLAGS.num_classes]),
                dtype=tf.float32)

            # scale and shift for layernorm
            self.gain = tf.Variable(tf.ones(
                [FLAGS.num_hidden_units]),
                dtype=tf.float32)
            self.bias = tf.Variable(tf.zeros(
                [FLAGS.num_hidden_units]),
                dtype=tf.float32)

        # fast weights and hidden state initialization
        self.A = tf.zeros(
            [FLAGS.batch_size, FLAGS.num_hidden_units, FLAGS.num_hidden_units],
            dtype=tf.float32)
        self.h = tf.zeros(
            [FLAGS.batch_size, FLAGS.num_hidden_units],
            dtype=tf.float32)

        # NOTE:inputs are batch-major
        # Process batch by time-major
        for t in range(0, FLAGS.input_dim):

            # hidden state (preliminary vector)
            self.h = tf.nn.relu((tf.matmul(self.X[:, t, :], self.W_x)+self.b_x) +
                (tf.matmul(self.h, self.W_h)))

            # Forward weight and layer normalization
            if FLAGS.model_name == 'RNN-LN-FW':

                # Reshape h to use with a
                self.h_s = tf.reshape(self.h,
                    [FLAGS.batch_size, 1, FLAGS.num_hidden_units])

                # Create the fixed A for this time step
                self.A = tf.add(tf.scalar_mul(self.l, self.A),
                    tf.scalar_mul(self.e, tf.batch_matmul(tf.transpose(
                        self.h_s, [0, 2, 1]), self.h_s)))

                # Loop for S steps
                for _ in range(FLAGS.S):
                    self.h_s = tf.reshape(
                        tf.matmul(self.X[:, t, :], self.W_x)+self.b_x,
                        tf.shape(self.h_s)) + tf.reshape(
                        tf.matmul(self.h, self.W_h), tf.shape(self.h_s)) + \
                        tf.batch_matmul(self.h_s, self.A)

                    # Apply layernorm
                    mu = tf.reduce_mean(self.h_s, reduction_indices=2) # each sample
                    sigma = tf.sqrt(tf.reduce_mean(tf.square(self.h_s - mu),
                        reduction_indices=2))
                    self.h_s = tf.div(tf.mul(self.gain, (self.h_s - mu)), sigma) + \
                        self.bias

                    # Apply nonlinearity
                    self.h_s = tf.nn.relu(self.h_s)

                # Reshape h_s into h
                self.h = tf.reshape(self.h_s,
                    [FLAGS.batch_size, FLAGS.num_hidden_units])

            elif FLAGS.model_name == 'RNN-LN': # no fast weights but still LN

                # Apply layer norm
                with tf.variable_scope('just_ln') as scope:
                    if t > 0:
                        scope.reuse_variables()
                    self.h = ln(self.h, scope='h/')

            elif FLAGS.model_name == 'CONTROL': # no fast weights or LN
                pass

        # All inputs processed! Time for softmax
        self.logits = tf.matmul(self.h, self.W_softmax) + self.b_softmax

        # Loss
        self.loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(self.logits, self.y))

        # Optimization
        self.lr = tf.Variable(0.0, trainable=False)
        self.trainable_vars = tf.trainable_variables()
        # clip the gradient to avoid vanishing or blowing up gradients
        self.grads, self.norm = tf.clip_by_global_norm(
            tf.gradients(self.loss, self.trainable_vars), FLAGS.max_gradient_norm)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.update = optimizer.apply_gradients(
            zip(self.grads, self.trainable_vars))

        # Accuracy
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.logits, 1),
            tf.argmax(self.y, 1)), tf.float32))

        # Components for model saving
        self.global_step = tf.Variable(0, trainable=False) # won't step
        self.saver = tf.train.Saver(tf.all_variables())

    def step(self, sess, batch_X, batch_y, l, e, forward_only=True):
        """
        Get results for training/validation.
        """
        input_feed = {self.X: batch_X, self.y: batch_y, self.l:l, self.e:e}

        if not forward_only: # training
            output_feed = [self.loss, self.accuracy, self.norm,
            self.update]
        elif forward_only: # validation
            output_feed = [self.loss, self.accuracy]

        # process outputs
        outputs = sess.run(output_feed, input_feed)

        if not forward_only:
            return outputs[0], outputs[1], outputs[2], outputs[3]
        elif forward_only:
            return outputs[0], outputs[1]


In [51]:
FLAGS = parameters()

In [52]:
def build_model(inp_shape):
    model = Sequential(name="FW-RNN")
    model.add(layers.InputLayer(input_shape=inp_shape))
    model.add(FW_RNN(FLAGS))
    model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    return model

In [53]:
cnn_FWrnn = build_model(inp_shape)
cnn_FWrnn.summary()

TypeError: Input() got multiple values for argument 'shape'

# Train + Evaluate model

In [ ]:
# AW2_norm_minitrain = AW2_norm_minitrain.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
history_best = cnn_FWrnn.fit(
    seq_train_features,
    seq_train_labels,
    sample_weight=samples_weights,
    validation_data=(seq_val_features, seq_val_labels),
    epochs=150,
    verbose=1,
)

# Predict on test set
1. Create loop which reads feature_data from each video directory
2. Predict on these features
3. Write predictions to filename.txt